<a href="https://colab.research.google.com/github/Sri-Raksha/GoogleNet/blob/main/E0321062_GoogLeNet_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from keras.models import Sequential, Model
import numpy as np
from keras import utils

In [ ]:
from keras.layers import Conv2D, MaxPooling2D, concatenate, Input, Dropout, Dense, AveragePooling2D, Flatten, GlobalAveragePooling2D

In [ ]:
from keras.datasets import mnist

In [ ]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

11490434/11490434 [==============================] - 0s 0us/step


In [ ]:
x_train.shape

(60000, 28, 28)

In [ ]:
x_train = np.expand_dims(x_train, axis=-1)
x_train = x_train.astype('float32') / 255
y_train = utils.to_categorical(y_train, num_classes=10)

In [ ]:
x_train.shape

(60000, 28, 28, 1)

In [ ]:
def add_inception_module(layer_no, prev_layer, filter1, filter5_in, filter5_out, filter3_in, filter3_out, pool_out):
    conv1 = Conv2D(filters=filter1, kernel_size=(1, 1), padding='same', activation='relu', name='inc-%d-1x1'%layer_no)(prev_layer)

    conv3_red = Conv2D(filters=filter3_in, kernel_size=(1, 1), padding='same', activation='relu')(prev_layer)
    conv3 = Conv2D(filters=filter3_out, kernel_size=(3, 3), padding='same', activation='relu', name='inc-%d-3x3'%layer_no)(conv3_red)

    conv5_red = Conv2D(filters=filter5_in, kernel_size=(1, 1), padding='same', activation='relu')(prev_layer)
    conv5 = Conv2D(filters=filter5_out, kernel_size=(5, 5), padding='same', activation='relu', name='inc-%s-5x5'%layer_no)(conv5_red)

    pool = MaxPooling2D((3,3), strides=(1,1), padding='same')(prev_layer)
    pool = Conv2D(pool_out, (1,1), padding='same', activation='relu', name='inc-%d-pool'%layer_no)(pool)

    return concatenate([conv1, conv3, conv5, pool], axis=-1)

In [ ]:
def get_branch_output(prev_layer, branch_no):
  branch = AveragePooling2D((3, 3), strides=3, name='branch_%d_pool'%branch_no)(prev_layer)
  branch = Conv2D(32, (1, 1), padding='same', activation='relu', name='branch_%d_conv'%branch_no)(branch)
  branch = Flatten(name='branch_%d_flatten'%branch_no)(branch)
  branch = Dropout(0.7, name='branch_%d_dropout'%branch_no)(branch)
  branch = Dense(10, activation='softmax', name='branch_%d_output'%branch_no)(branch)
  return branch

In [ ]:
input_layer = Input(shape=(28, 28, 1))

In [ ]:
input_layer.shape

TensorShape([None, 28, 28, 1])

In [ ]:
first_layer = Conv2D(filters=64, kernel_size=(7, 7), padding='same', strides=(2, 2), activation='relu', name='first_7x7')(input_layer)
first_layer = MaxPooling2D(pool_size=(3, 3), padding='same', strides=(2, 2), name='first_pool')(first_layer)
first_layer = Conv2D(filters=64, kernel_size=(1, 1), padding='same', strides=(1, 1), activation='relu', name='first_conv_3x3')(first_layer)
first_layer = Conv2D(filters=192, kernel_size=(3, 3), padding='same', strides=(1, 1), activation='relu', name='first_conv_3x3_b/1')(first_layer)
first_layer = MaxPooling2D(pool_size=(3, 3), padding='same', strides=(2, 2), name='first_pool/2')(first_layer)

In [ ]:
first_layer.shape

TensorShape([None, 4, 4, 192])

In [ ]:
inc = add_inception_module(layer_no=1,
                           prev_layer=first_layer,
                           filter1=64,
                           filter3_in=96,
                           filter3_out=128,
                           filter5_in=16,
                           filter5_out=32,
                           pool_out=32)

In [ ]:
inc2 = add_inception_module(layer_no=2,
                            prev_layer=inc,
                            filter1=128,
                            filter3_in=128,
                            filter3_out=192,
                            filter5_in=32,
                            filter5_out=96,
                            pool_out=64)

In [ ]:
inc.shape

TensorShape([None, 4, 4, 256])

In [ ]:
inc2.shape

TensorShape([None, 4, 4, 480])

In [ ]:
inc3 = add_inception_module(layer_no=3,
                            prev_layer=inc2,
                            filter1=192,
                            filter3_in=96,
                            filter3_out=208,
                            filter5_in=16,
                            filter5_out=48,
                            pool_out=64)

In [ ]:
branch1 = get_branch_output(prev_layer=inc3, branch_no=1)

In [ ]:
inc4 = add_inception_module(layer_no=4,
                            prev_layer=inc3,
                            filter1=160,
                            filter3_in=112,
                            filter3_out=224,
                            filter5_in=24,
                            filter5_out=64,
                            pool_out=64)

In [ ]:
inc5 = add_inception_module(layer_no=5,
                            prev_layer=inc4,
                            filter1=128,
                            filter3_in=128,
                            filter3_out=256,
                            filter5_in=24,
                            filter5_out=64,
                            pool_out=64)

In [ ]:
inc6 = add_inception_module(layer_no=6,
                            prev_layer=inc5,
                            filter1=112,
                            filter3_in=144,
                            filter3_out=288,
                            filter5_in=32,
                            filter5_out=64,
                            pool_out=64)

In [ ]:
branch2 = get_branch_output(prev_layer=inc6, branch_no=2)

In [ ]:
inc7 = add_inception_module(layer_no=7,
                            prev_layer=inc6,
                            filter1=256,
                            filter3_in=160,
                            filter3_out=320,
                            filter5_in=32,
                            filter5_out=128,
                            pool_out=128)

In [ ]:
inc8 = add_inception_module(layer_no=8,
                            prev_layer=inc7,
                            filter1=384,
                            filter3_in=192,
                            filter3_out=384,
                            filter5_in=48,
                            filter5_out=128,
                            pool_out=128)

In [ ]:
out = GlobalAveragePooling2D()(inc8)
out = Dropout(0.7)(out)
out = Dense(10, activation='softmax', name='final_output')(out)

In [ ]:
model = Model(input_layer, [out, branch1, branch2])

In [ ]:
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 28, 28, 1)]          0         []                            
                                                                                                  
 first_7x7 (Conv2D)          (None, 14, 14, 64)           3200      ['input_1[0][0]']             
                                                                                                  
 first_pool (MaxPooling2D)   (None, 7, 7, 64)             0         ['first_7x7[0][0]']           
                                                                                                  
 first_conv_3x3 (Conv2D)     (None, 7, 7, 64)             4160      ['first_pool[0][0]']          
                                                                                            

In [ ]:
model.build(input_shape=x_train.shape)

In [ ]:
model.compile(optimizer="sgd", loss="categorical_crossentropy", metrics=['accuracy'])

In [ ]:
x_train.shape

(60000, 28, 28, 1)

In [ ]:
model.fit(x_train, y_train, epochs=100)

Epoch 1/100
1875/1875 [==============================] - 996s 529ms/step - loss: 6.3763 - final_output_loss: 2.0383 - branch_1_output_loss: 2.1599 - branch_2_output_loss: 2.1781 - final_output_accuracy: 0.2404 - branch_1_output_accuracy: 0.1989 - branch_2_output_accuracy: 0.1716
Epoch 2/100
1875/1875 [==============================] - 986s 526ms/step - loss: 2.2295 - final_output_loss: 0.3776 - branch_1_output_loss: 0.9424 - branch_2_output_loss: 0.9094 - final_output_accuracy: 0.8794 - branch_1_output_accuracy: 0.6511 - branch_2_output_accuracy: 0.6556
Epoch 3/100
1875/1875 [==============================] - 987s 526ms/step - loss: 1.1791 - final_output_loss: 0.1136 - branch_1_output_loss: 0.5654 - branch_2_output_loss: 0.5002 - final_output_accuracy: 0.9683 - branch_1_output_accuracy: 0.7835 - branch_2_output_accuracy: 0.8088
Epoch 4/100
1875/1875 [==============================] - 979s 522ms/step - loss: 0.9776 - final_output_loss: 0.0734 - branch_1_output_loss: 0.4731 - branch_2_ou